### TODO

1. lambda functions to create quote and order notification messages
2. implement threads for handling live streaming
  - Design json message structure for each handler
  - Implement decoder for json messages for each handler
3. Implement trade job - init, scanner, order
4. Implement user request handler


#### Future
1. Define function to restart a stopped thread
2. Watchdog implementation to resume processes
3. Implementation of user initiated aborts and restart

## Thread functions

In [1]:
from lib.multitasking_lib import *
#from lib.kite_helper_lib import *
import sys

In [2]:
# Long Strategies
def long_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) <=20)&(s.fastk>20)
    #temp_df = temp_df | CROSSOVER(s.fastk, s.fastd)
    #temp_df = temp_df | (REF(s.macd,1) <=0)&(s.macd>0)
    #temp_df = temp_df | CROSSOVER(s.macd, s.macdsignal)
    temp_df = temp_df | (REF(s.rsi,1) <=40)
    #temp_df = temp_df | (s.CLOSE.shift(1) == s.min.shift(1)) & (s.CLOSE > s.OPEN)
    #temp_df = temp_df & CROSSOVER(s.BBB, s.CLOSE)

    return pd.DataFrame(  temp_df, columns=["buy"] )

# Short Strategies
def short_indicators(s):
    temp_df = 0
    #temp_df = temp_df | (REF(s.fastk,1) >=80)&(s.fastk<80)
    #temp_df = temp_df | CROSSOVER(s.fastd, s.fastk)
    #temp_df = temp_df | (REF(s.macd,1) >=0)&(s.macd<0)
    #temp_df = temp_df | CROSSOVER(s.macdsignal, s.macd)
    temp_df = temp_df | (REF(s.rsi,1) >=60)
    #temp_df = temp_df & CROSSOVER(s.CLOSE, s.BBT)
    return pd.DataFrame( temp_df, columns=["sell"])



def long_breakout(s):
    temp_df = s.haCLOSE >= s.BBT.shift(1)
    temp_df = temp_df | (s.haCLOSE >= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.OPEN, s.BBT) | CROSSOVER(s.OPEN, s.BBB) | CROSSOVER(s.OPEN, s.BBM))
    return pd.DataFrame( temp_df , columns=["buy"])

def short_breakout(s):
    temp_df = s.haOPEN <= s.BBB.shift(1)
    temp_df = temp_df | (s.haCLOSE <= s.haOPEN.shift(2))
    temp_df = temp_df & ( CROSSOVER(s.BBT, s.OPEN) | CROSSOVER(s.BBB, s.OPEN) | CROSSOVER(s.BBM, s.OPEN))
    return pd.DataFrame( temp_df , columns=["sell"])

def long_ha(s):
    temp_df = (REF(s.haCLOSE,3) < REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) < REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) > REF(s.haOPEN,1)) & (s.haCLOSE > s.haOPEN)  
    #temp_df = temp_df & (s.rsi < 40 )
    return pd.DataFrame( temp_df , columns=["buy"])

def short_ha(s):
    temp_df = (REF(s.haCLOSE,3) > REF(s.haOPEN,3)) & (REF(s.haCLOSE,2) > REF(s.haOPEN,2)) & (REF(s.haCLOSE,1) < REF(s.haOPEN,1)) & (s.haCLOSE < s.haOPEN)
    #temp_df = temp_df & (s.rsi > 60 )
    return pd.DataFrame( temp_df , columns=["sell"])
    


In [3]:
# ====== Tradescript Wrapper =======
# Methods

TREND_UP = lambda a,b: ROC(a, b) >= 0.1
TREND_DOWN = lambda a,b: ROC(a, b) <= -0.1

import traceback
#TREND_UP = lambda a,b: a > MAX(REF(a,1),b)
#TREND_DOWN = lambda a,b: a < MIN(REF(a,1),b)

CROSSOVER = lambda a, b: (REF(a,1)<=REF(b,1)) & (a > b)

#Heikin Asi
def HAIKINASI(ohlc_data_df):
    REF = lambda key, i: ohlc_get(ohlc_data_df.shift(i), key)
    
    OPEN  = ohlc_data_df['open']
    HIGH  = ohlc_data_df['high']
    LOW   = ohlc_data_df['low']
    CLOSE = ohlc_data_df['close']
    
    haOPEN  = (OPEN.shift(1) + CLOSE.shift(1))/2
    haHIGH  = pd.DataFrame([HIGH,OPEN,CLOSE]).max(axis = 0, skipna = True)
    haLOW   = pd.DataFrame([HIGH,LOW,CLOSE]).min(axis = 0, skipna = True)
    haCLOSE = (OPEN+HIGH+LOW+CLOSE)/4
    
    return (haOPEN, haHIGH, haLOW, haCLOSE)

ohlc_get = lambda df, key: df.iloc[-1][key]
REF = lambda df, i: df.iloc[-i-1]
def myalgo(ohlc_data_df): 
    ohlc_data_temp = ohlc_data_df.tail(30)
    
    OPEN = ohlc_data_temp['open']
    CLOSE = ohlc_data_temp['close']
    HIGH = ohlc_data_temp['high']
    LOW = ohlc_data_temp['low']
    VOLUME = ohlc_data_temp['volume']
    
    (haOPEN, haHIGH, haLOW, haCLOSE) = HAIKINASI(ohlc_data_temp)
    
    if (REF(haCLOSE,1) < REF(haOPEN,1)) and (REF(haCLOSE,0) > REF(haOPEN,0)): 
        return "BUY"
    elif (REF(haCLOSE,1) > REF(haOPEN,1)) and (REF(haCLOSE,0) < REF(haOPEN,0)): 
        return "SELL"
    
    return "WAIT" #"BUY"|"SELL"

#ohlc_df = pd.read_json(conn.hget(stock+'_state','ohlc'))
#myalgo(ohlc_df)

In [4]:
# Shut down
conn.publish('kite_simulator','stop')
conn.publish('trade_handler','stop')
freedom.job.terminate()
print(freedom.job.is_alive())
conn.pubsub_channels()

NameError: name 'freedom' is not defined

## Definition of handler functions

In [6]:

def trade_handler(manager, msg):
    pdebug('trade_handler: {}'.format(msg))
    # Step 1: Blocking call to msgBufferQueue and notificationQueue
    conn.xtrim('msgBufferQueue',maxlen=0, approximate=False)
    conn.xtrim('notificationQueue',maxlen=0, approximate=False)
    while(True):
        msg_q = conn.xread({'msgBufferQueue':'$','notificationQueue':'$'}, block=0, count=100)
        msgs_q = conn.xread({'msgBufferQueue':'0','notificationQueue':'0'}, block=1, count=100)
        conn.xtrim('msgBufferQueue',maxlen=0, approximate=False)
        conn.xtrim('notificationQueue',maxlen=0, approximate=False)
        
        # Step 2: Process notifications: Start a worker thread for each notification
        
        #TODO
        
        # Step 3: Process tick: Start a worker thread for each msg        
        for msg in msgs_q[0][1]:
            pdebug('trade_handler: {}'.format(msg[1]['msg']))
            
            try:
                data = json.loads(msg[1]['msg'])
            except:
                perror("Un-supported message: {} : {}".format(msg, sys.exc_info()[0]))
                break

            for key in data.keys():
                stock = key.split(':')[1]
                exchange = key.split(':')[0]

            hash_key = stock+'_state'
            freq = conn.hget(hash_key,'freq')
            state = conn.hget(hash_key,'state')


            temp_df = msg_to_ohlc(data)
            if state == 'INIT': # State: Init: Load historical data from cache
                # 1: Populate Redis buffer stock+"OHLCBuffer" with historical data
                toDate = (temp_df.index[0] - timedelta(days=1)).strftime('%Y-%m-%d')
                fromDate = (temp_df.index[0] - timedelta(days=no_of_hist_candles)).strftime('%Y-%m-%d')
                ohlc_data = getData(stock, fromDate, toDate, exchange, freq, False, stock)
            else: # Load data from OHLC buffer in hash
                ohlc_data = pd.read_json(conn.hget(hash_key, 'ohlc'))

            ohlc_data = ohlc_data.append(temp_df)
            conn.hset(hash_key,'ohlc',ohlc_data.to_json())

            # Add to OHLCBuffer in hash
            manager.add(stock, trade_job, False, hash_key)
            pdebug(msg[0])
            
# A thread function to process notifications and tick
algo_idle = myalgo
algo_long_so = myalgo
algo_short_so = myalgo

trade_lock = Lock() #TODO: make lock per stockname
def trade_job(hash_key):
    pdebug('trade_job: {}'.format(hash_key))
    
    trade_lock.acquire()
    # Step 1.1: Get stock name from the message    
    
    # Step 1.2: Get state for the stock from the redis
    state = conn.hget(hash_key,'state')
    if not state:
        return
    stock = conn.hget(hash_key,'stock')
    freq = conn.hget(hash_key,'freq')
    pdebug("{}: {}: {}".format(hash_key, stock, state ))
    ohlc_df = pd.read_json(conn.hget(hash_key,'ohlc'))
    
    last_processed = ohlc_df.index[-1].strftime('%Y-%m-%d')
    pdebug("{}=>{}".format(last_processed,conn.hget(hash_key,'last_processed')))
    
    if last_processed == conn.hget(hash_key,'last_processed'):   
        trade_lock.release()
        return
    else:
        conn.hset(hash_key,'last_processed',last_processed)
    
    #print('{}:{}'.format(stock,ohlc_df.index[-1]))
    
    # Step 2: Switch to appropriate state machine based on current state
    if state == 'INIT': # State: Init
        # 1: Populate Redis buffer stock+"OHLCBuffer" with historical data
            # Done inside thread handler
        
        # 2: Set state to Scanning
        conn.hset(hash_key,'state','SCANNING')
        pass
    
    elif state == 'SCANNING':  # State: Scanning
        # 1: Run trading algorithm for entering trade
        tradeDecision = algo_idle(ohlc_df)
        
        # 2: If Algo returns Buy: set State to 'Pending Order: Long'
        if tradeDecision=="BUY":
            pwarning("BUY: {} -> {}".format(stock, ohlc_get(ohlc_df,'close')))
            conn.hset(hash_key,'state','PO:LONG')
        
        # 3: If Algo returns Sell: set State to 'Pending Order: Short'
        elif tradeDecision=="SELL":
            pwarning("SELL: {} -> {}".format(stock, ohlc_get(ohlc_df,'close')))
            conn.hset(hash_key,'state','PO:SHORT')
        
        # 4: Update TradeMetaData: Push order details to OrderQueue
    
    elif state == 'PO:LONG': # State: Pending Order: Long
    
        # 1: On Fill: set State to Long
        conn.hset(hash_key,'state','LONG')
        pass
    
    
    elif state == 'PO:SHORT': # State: Pending Order: Short
    
        # 1: On Fill: set State to Short
        conn.hset(hash_key,'state','SHORT')
        pass
    
    
    elif state == 'LONG': # State: Long
    
        # 1: If notification for AutoSquare Off: set state to init
        
        # 2: Else run trading algorithm for square off
        
        tradeDecision = algo_long_so(ohlc_df)
        if tradeDecision == "SELL":
            pwarning("SO-S: {} -> {}".format(stock, ohlc_get(ohlc_df,'close')))

            # 3: If algo returns square off: then push square off details to OrderQueue, set state to 'Awaiting Square Off'   
            conn.hset(hash_key,'state','SQUAREOFF')
    
    
    elif state == 'SHORT': # State: Short
    
        # 1: If notification for AutoSquare Off: set state to init
        
        # 2: Else run trading algorithm for square off
        tradeDecision = algo_short_so(ohlc_df)
        
        if tradeDecision == "BUY":
            pwarning("SO-B: {} -> {}".format(stock, ohlc_get(ohlc_df,'close')))
        
            # 3: If algo returns square off: then push square off details to OrderQueue, set state to 'Awaiting Square Off'
    
            conn.hset(hash_key,'state','SQUAREOFF')
        
    elif state == 'SQUAREOFF':  # State: Awaiting Square Off
        
        conn.hset(hash_key,'state','INIT')
        pass
   
        # 1: On Fill notification: set state to Init

    trade_lock.release()



In [7]:
#freedom_init = threadManager("freedom_init", ["freedom_init"], [freedom_init])   

#freedom = threadManager("freedom", ["user_requests_handler", "kite_simulator", "backtest_handler", "trade_handler","order_handler"], 
#                        [user_requests_handler, kite_simulator, backtest_handler, trade_handler, order_handler])

logger.setLevel(logging.WARNING)
freedom = threadManager("freedom", [ "kite_simulator", "trade_handler", "order_handler"], 
                        [kite_simulator, trade_handler, order_handler])


INFO:	NumExpr defaulting to 2 threads.
ERROR:	Un-supported message: ('1587242206555-0', {'msg': 'msg'}) : <class 'json.decoder.JSONDecodeError'>
ERROR:	Un-supported message: ('1587242218695-0', {'msg': 'stop'}) : <class 'json.decoder.JSONDecodeError'>


In [8]:

# Unit Test#1
freq='day'
exchange='NSE'
symbol='WIPRO'
fromDate = '2019-01-01'
toDate = '2019-05-03'
stock=symbol

state = 'INIT'
hash_key = stock+'_state'

try:
    all_keys = list(conn.hgetall(hash_key).keys())
    conn.hdel(hash_key,*all_keys)
except:
    pass

#conn.hmset(hash_key, {'state':state,'stock':stock, 'qty':0,'price':0,'algo':'','freq':'day','so':0,'target':0})

logger.setLevel(logging.INFO)
msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)


In [13]:
# Shut down
conn.publish('kite_simulator','stop')
conn.publish('trade_handler','stop')
freedom.job.terminate()
print(freedom.job.is_alive())
conn.pubsub_channels()

False


['backtest']

In [11]:
notification_despatcher(None, 'stop')

In [9]:
# Unit Test#2
stock = 'TCS'
msg = json.dumps({'stock': stock, 'fromDate':fromDate,'toDate':toDate, 'exchange':exchange, 'freq':freq})
conn.publish('trade_handler','start')
msgid2 = conn.publish('kite_simulator',msg)
notification_despatcher(None, 'msg')

In [10]:
conn.pubsub_channels()

['trade_handler', 'backtest', 'kite_simulator', 'order_handler']

In [34]:
# Debug
stock='WIPRO'
#notification_despatcher(None, 'msg')
print(stock)
pd.read_json(conn.hget(stock+'_state','ohlc'))

WIPRO


,close,high,low,open,volume
2018-09-24,250.15,254.14,248.31,248.31,3740868
2018-09-25,246.84,251.80,245.30,250.38,3389131
2018-09-26,240.04,247.33,237.89,246.77,7864825
2018-09-27,239.81,243.35,238.12,242.03,17997289
2018-09-28,243.61,245.60,237.14,239.85,6912444
...,...,...,...,...,...
2019-04-04,259.05,262.25,254.95,261.00,6419198
2019-04-05,262.10,263.85,259.50,261.00,3614552
2019-04-08,263.70,265.50,260.75,263.70,3637464
2019-04-09,273.80,275.00,263.35,263.50,13991384


In [10]:
pd.set_option('display.max_rows', 50)
pd.read_json(conn.hget(stock+'_state','ohlc')).tail(50)

,close,high,low,open,volume
2019-02-15,282.78,283.72,278.31,282.14,3522673
2019-02-18,282.41,284.81,278.76,284.81,3480075
2019-02-19,273.27,282.26,272.33,280.11,3064358
2019-02-20,279.85,281.20,271.58,273.08,4410484
2019-02-21,281.62,283.31,278.80,280.30,2573085
2019-02-22,284.77,285.71,280.83,281.95,3194664
2019-02-25,289.77,290.75,285.15,286.32,4678470
2019-02-26,291.47,292.44,286.65,289.10,3802537
2019-02-27,281.50,288.61,280.00,285.90,8009017
2019-02-28,277.37,283.23,275.64,281.88,9428265


## Scratchpad

In [ ]:

 
if False:
    msg_ohlc = {"NSE:WIPRO": {"ohlc": {"date": "2019-1-1", "open": 1896.0, "high": 1910.0, "low": 1885.0, "close": 1902.8, "volume": 1094883.0}}}

    stock='WIPRO'
    state = 'INIT'
    hash_key = stock+'_state'

    try:
        all_keys = list(conn.hgetall(hash_key).keys())
        conn.hdel(hash_key,*all_keys)
    except:
        pass

    conn.hmset(hash_key, {'state':state,'stock':stock, 'qty':0,'price':0,'algo':'','freq':'day','so':0,'target':0})

    logger.setLevel(logging.INFO)
    trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))
    #trade_job(json.dumps(msg_ohlc))

if False:
    msg_ohlc = {"NSE:WIPRO": {"ohlc": {"date": "2019-1-3", "open": 1896.0, "high": 1910.0, "low": 1885.0, "close": 1902.8, "volume": 1094883.0}}}
    trade_job(json.dumps(msg_ohlc))